In [1]:
# Dependencies

import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset

c:\Users\Willson\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### ENCODER PART
1. DATA LOADING
2. IMAGE PREPROCESSING, RESIZING, CLEANING
3. VISION TRANSFORMER ENCODER
*   patch embeddding
*   vision transformer
4. TRAINING LOOP

DATA LOADING OF POLYVORE SET
Do we want set to be categorized beforehand or not?

In [2]:
directory = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images'

for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    print(file_path)
    if os.path.isfile(file_path):  # Check if it's a file
        print(file_path)

c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\bag
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\bracelet
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\brooch
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\dress
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\earrings
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\eyewear
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\gloves
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\hairwear
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\hats
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\jumpsuit
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\legwear
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\necklace
c:/Users/Willson/Docum

In [6]:
# Split into training, validation sets

import os
import shutil
import random
from pathlib import Path

# Parameters
data_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images'  # The root directory containing your category folders
output_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split'   # The directory where the split dataset will be stored
val_split = 0.2                 # Proportion of the dataset to include in the validation split

# Ensure the output directories exist
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Function to split the dataset
def split_dataset(data_dir, train_dir, val_dir, val_split):
    categories = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    
    for category in categories:
        category_path = os.path.join(data_dir, category)
        images = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        
        random.shuffle(images)
        val_size = int(len(images) * val_split)
        
        train_images = images[val_size:]
        val_images = images[:val_size]
        
        train_category_dir = os.path.join(train_dir, category)
        val_category_dir = os.path.join(val_dir, category)
        
        os.makedirs(train_category_dir, exist_ok=True)
        os.makedirs(val_category_dir, exist_ok=True)
        
        for img in train_images:
            shutil.copy2(os.path.join(category_path, img), os.path.join(train_category_dir, img))
        
        for img in val_images:
            shutil.copy2(os.path.join(category_path, img), os.path.join(val_category_dir, img))
        
        print(f'Category: {category}')
        print(f'Total images: {len(images)}')
        print(f'Training images: {len(train_images)}')
        print(f'Validation images: {len(val_images)}')

# Execute the split
split_dataset(data_dir, train_dir, val_dir, val_split)


Category: bag
Total images: 21270
Training images: 17016
Validation images: 4254
Category: bracelet
Total images: 5175
Training images: 4140
Validation images: 1035
Category: brooch
Total images: 996
Training images: 797
Validation images: 199
Category: dress
Total images: 7480
Training images: 5984
Validation images: 1496
Category: earrings
Total images: 5509
Training images: 4408
Validation images: 1101
Category: eyewear
Total images: 6681
Training images: 5345
Validation images: 1336
Category: gloves
Total images: 387
Training images: 310
Validation images: 77
Category: hairwear
Total images: 693
Training images: 555
Validation images: 138
Category: hats
Total images: 2914
Training images: 2332
Validation images: 582
Category: jumpsuit
Total images: 297
Training images: 238
Validation images: 59
Category: legwear
Total images: 203
Training images: 163
Validation images: 40
Category: necklace
Total images: 4665
Training images: 3732
Validation images: 933
Category: neckwear
Total ima

IMAGE PREPROCESSING

Resizing down to 256x256 size, or maybe smaller, depending on training speed and quality

VISION TRANSFORMER ENCODER - pre trained model

In [2]:
from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer

# Load pre-trained ViT, feature extractor
# num_classes = len(os.listdir(os.path.join(data_dir, 'train')))
model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

c:\Users\Willson\anaconda3\envs\ml\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [14]:
# PIL time
import numpy as np
from PIL import Image

image = Image.open('c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images/bag/10014953_9.jpg')
np.asarray(image)

array([[[251, 251, 251],
        [251, 251, 251],
        [251, 251, 251],
        ...,
        [251, 251, 251],
        [251, 251, 251],
        [251, 251, 251]],

       [[251, 251, 251],
        [251, 251, 251],
        [251, 251, 251],
        ...,
        [251, 251, 251],
        [251, 251, 251],
        [251, 251, 251]],

       [[251, 251, 251],
        [251, 251, 251],
        [251, 251, 251],
        ...,
        [251, 251, 251],
        [251, 251, 251],
        [251, 251, 251]],

       ...,

       [[251, 251, 251],
        [251, 251, 251],
        [251, 251, 251],
        ...,
        [251, 251, 251],
        [251, 251, 251],
        [251, 251, 251]],

       [[251, 251, 251],
        [251, 251, 251],
        [251, 251, 251],
        ...,
        [251, 251, 251],
        [251, 251, 251],
        [251, 251, 251]],

       [[251, 251, 251],
        [251, 251, 251],
        [251, 251, 251],
        ...,
        [251, 251, 251],
        [251, 251, 251],
        [251, 251, 251]]

TRAINING LOOP

In [25]:
# Hyperparameters
batch_size = 64
num_epochs = 10
learning_rate = 5e-5

# Data processing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets, DataLoaders
data_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split'
train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Convert for transformers
class ImageDataset(Dataset):
    def __init__(self, image_folder):
        self.image_folder = image_folder
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        image, label = self.image_folder[idx]
        image = self.feature_extractor(images=image, return_tensors="pt")['pixel_values'][0]
        return {'pixel_values': image, 'labels': label}

train_dataset = ImageDataset(train_dataset)
val_dataset = ImageDataset(val_dataset)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


In [26]:
# Train
training_args = TrainingArguments(
    output_dir=os.path.join(data_dir, 'results'),
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    eval_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor
)

trainer.train()

model.save_pretrained('devito')
feature_extractor.save_pretrained('devito')

  0%|          | 0/15870 [00:40<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs[0], labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

### RECOMMENDER PART

VECTOR EXTRACTION

In [ ]:
# Extracting features (latent vectors) from the trained Vision Transformer
def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model.patch_embed(inputs)  # Get patch embeddings
            cls_tokens = model.cls_token.expand(inputs.size(0), -1, -1)
            x = torch.cat((cls_tokens, outputs), dim=1)
            x = x + model.pos_embed
            x = model.pos_drop(x)
            x = model.transformer(x)
            x = model.norm(x[:, 0])
            features.append(x.cpu().numpy())
    return np.concatenate(features)

# Data loader for feature extraction
feature_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=4)
item_features = extract_features(model, feature_loader)

USER PROFILE AND RECOMMENDATION (currently ML, could be upgraded to a deep learning model based on user specifications)

In [ ]:
# Example: User interaction data
user_interactions = {
    'user1': [0, 1, 2],  # Indices of items the user interacted with
    'user2': [3, 4, 5],
    # ...
}

# Create user profiles based on average features of interacted items
user_profiles = {
    user_id: np.mean([item_features[item_id] for item_id in items], axis=0)
    for user_id, items in user_interactions.items()
}

# Recommendation function
from sklearn.metrics.pairwise import cosine_similarity

def recommend(user_id, top_n=10):
    user_vector = user_profiles[user_id].reshape(1, -1)
    similarities = cosine_similarity(user_vector, item_features)
    recommended_items = np.argsort(similarities[0])[-top_n:]
    return recommended_items

# Example usage
user_id = 'user1'
recommended_items = recommend(user_id)
print(f"Recommended items for user {user_id}: {recommended_items}")